In [39]:
import numpy as np
import pandas as pd
import sqlite3
import sys
import sqlalchemy
from ipywidgets import *
import ipywidgets as widgets
from IPython.display import display
from traitlets import link

# 1 Database Selection

In [44]:
database_frequencies = widgets.Dropdown(
    options=['AVA_FY', 'AVA_CY', 'AVA_FQ', 'AVA_CQ'],
    value='AVA_FY',
    description="Selected Database: ",
)
display(database_frequencies)

In [45]:
database_frequencies.value

'AVA_FY'

In [46]:
#Loading database after selection
## Connecting database
db_conn = sqlite3.connect(database_frequencies.value+'.db')
db_cur = db_conn.cursor()
print ("Database "+ database_frequencies.value +" Connection Established")

Database AVA_FY Connection Established


## 1.1 Database loading

In [47]:
#load all tables
db_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
metric_tables = db_cur.fetchall()
metric_names = [i[0] for i in metric_tables]
var_names = [name.replace(" ", "_") for name in metric_names]
df_ava = {}
for i in range(len(var_names)):
    sql_query = 'SELECT * FROM "{table_name}"'.format(table_name=metric_names[i])
    key = metric_names[i]
    value = pd.read_sql(sql_query, db_conn)
    df_ava[key] = value

In [48]:
df_ava["EBITDA"].head()

,index,EBITDA,Company Name,Exchange:Ticker,Country,Region,Primary Industry,AV Industry,AV Sector,FY1998,...,FY2008,FY2009,FY2010,FY2011,FY2012,FY2013,FY2014,FY2015,FY2016,IQ_LTM
0,0,IQ_EBITDA,"Wal-Mart Stores, Inc. (NYSE:WMT)",NYSE:WMT,United States,North America,Hypermarkets and Super Centers,Services,Retail,8137,...,28496,29889,31663,33183,34688,36302,35742,36569,33559,33006
1,1,None,Toyota Motor Corporation (TSE:7203),TSE:7203,Japan,Asia-Pacific,Automobile Manufacturers,Auto & Transportation,Auto OEM,12153.15587,...,32886.08073,9041.43186,13656.97643,14371.8479,12444.98136,21996.82586,30975.38834,36265.38002,39166.00715,32128.88543
2,2,None,Volkswagen Aktiengesellschaft (XTRA:VOW3),XTRA:VOW3,Germany,Europe,Automobile Manufacturers,Auto & Transportation,Auto OEM,2760.06231,...,10854.28722,7629.6686,12999.47396,17576.9595,20436.61231,19825.35508,21605.47081,17198.72476,0,23204.62915
3,3,None,Royal Dutch Shell plc (ENXTAM:RDSA),ENXTAM:RDSA,Netherlands,Europe,Integrated Oil and Gas,Resources/Process Material,Oil & Gas,10029,...,56416,29079,41839,55943,52114,48379,44378,23453,0,22113
4,4,None,Apple Inc. (NasdaqGS:AAPL),NasdaqGS:AAPL,United States,North America,"Technology Hardware, Storage and Peripherals",TIME,Tech HW Equipment,379,...,6721,8336,19364,35574,58518,55756,60449,82487,70529,69750


# 2. Data Selection and Filtering

## 2.1 Select Multiple Metrics

In [49]:
financial_tables = widgets.SelectMultiple(
    description="Select Financial Metrics",
    options=metric_names)

display(financial_tables)

In [61]:
selected_metric = financial_tables.value

## 2.2 Select Industry, Sector, Region, Country, Year with Drop Down Boxes 

###  2.2.1 Select Industry

In [62]:
# Convert industry list to set
industry_list = df_ava["Total Revenue"]["AV Industry"]
industry_list = set(industry_list)
industry_list = list(industry_list)
industry_list

['Auto & Transportation',
 'Services',
 'TIME',
 'Engineering',
 'Resources/Process Material']

In [76]:
select_industry = widgets.Dropdown(
    options=industry_list,
    value='Auto & Transportation',
    description="Selected Industry: ",
)
display(select_industry)

###  2.2.2 Select Sector

In [77]:
# Convert sector list to set
sector_list = df_ava["Total Revenue"]["AV Sector"]
sector_list = set(sector_list)
sector_list = list(sector_list)
sector_list[:5]

['Construction', 'Aluminum', 'Online Retail', 'Publishing', 'Entertainment']

In [78]:
select_sector = widgets.Dropdown(
    options=list(np.sort(sector_list)),
    value='Auto OEM',
    description="Selected Sector: ",
)
display(select_sector)

## 2.2.3 Select Region

In [79]:
# Convert sector list to set
region_list = df_ava["Total Revenue"]["Region"]
region_list = set(region_list)
region_list = list(region_list)

In [80]:
select_region = widgets.Dropdown(
    options=region_list,
    value='North America',
    description="Selected Sector: ",
)
display(select_region)

### 2.2.4 Select Country

In [81]:
# Convert sector list to set
country_list = df_ava["Total Revenue"]["Country"]
country_list = set(country_list)
country_list = list(country_list)

In [82]:
select_country = widgets.Dropdown(
    options=country_list,
    value='United States',
    description="Selected Sector: ",
)
display(select_country)

### 2.2.5 Select Years

In [83]:
column_names = list(df_ava["Total Revenue"].columns)
year_list = column_names[column_names.index("FY1998"):]

In [85]:
select_years = widgets.SelectMultiple(
    description="Select Years To Be Included",
    options=year_list)

display(select_years)

## 3 Real World Data Manipulation
### Select All US Airlines Company and Display Their Total Revenue, COGS, SG&A, EBIT, EBITDA between 2010 and 2015

In [103]:
# Define info list about selected companies
info_list=["Company Name", "Exchange:Ticker", "Country", "Region", "AV Industry", "AV Sector"]
display_columns = info_list + list(select_years.value)

In [104]:
df_ava_selection={}

In [105]:
for table in financial_tables.value:
    global display_columns
    df_ava_selection[table]=df_ava[table][df_ava[table]["AV Sector"]==select_sector.value][df_ava[table]["Country"]==select_country.value]
    df_ava_selection[table]=df_ava_selection[table].loc[:,display_columns]

C:\PROGRAMDATA\ANACONDA3\lib\site-packages\ipykernel\__main__.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  app.launch_new_instance()


In [106]:
df_ava_selection["COGS Ratio"]

,Company Name,Exchange:Ticker,Country,Region,AV Industry,AV Sector,FY2011,FY2012,FY2013,FY2014,FY2015
150,Continental Aktiengesellschaft (DB:CON),DB:CON,Germany,Europe,Auto & Transportation,Tier 1,0.790296,0.784462,0.765936,0.748850,0.740640
1172,Hella KGaA Hueck & Co. (DB:HLE),DB:HLE,Germany,Europe,Auto & Transportation,Tier 1,0.736360,0.733844,0.730298,0.723590,0.732256
1515,LEONI AG (DB:LEO),DB:LEO,Germany,Europe,Auto & Transportation,Tier 1,0.816223,0.822778,0.827009,0.828855,0.839454
3204,Grammer Aktiengesellschaft (DB:GMM),DB:GMM,Germany,Europe,Auto & Transportation,Tier 1,0.869693,0.876145,0.876807,0.884118,0.893308
3328,ElringKlinger AG (DB:ZIL2),DB:ZIL2,Germany,Europe,Auto & Transportation,Tier 1,0.719454,0.722845,0.719993,0.729618,0.751681


In [102]:
export_button = widgets.Button(
    description='Export to Excel',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check'
)
display(export_button)

def export_button_click(click):
    export_to_excelfile = pd.ExcelWriter(select_sector.value+" "+select_country.value+" "+"Data Export.xlsx")
    
    for table in df_ava_selection:
        df_ava_selection[table].to_excel(export_to_excelfile, sheet_name=table, index=False)
    export_to_excelfile.save()

export_button.on_click(export_button_click)